In [1]:
%run util.py

In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

# Loading JSON data
with open(data_folder + 'jam_ref_data.json', 'r') as json_file:
    jam_ref_data = json.load(json_file)
    
with open(data_folder + 'jam_test_data.json', 'r') as json_file:
    jam_test_data = json.load(json_file)

### Quantize the features by k-means clustering

In [4]:
### Quantize the features by k-means clustering 

import numpy as np

loc_list = []
length_list = []
numPts_list = []
speed_list = []

ref_loc_list = []
ref_length_list = []
ref_numPts_list = []
ref_speed_list = []
for key in jam_ref_data.keys():
    ref_loc_list.append(jam_ref_data[key]['longitude_latitude'])
    loc_list.append(jam_ref_data[key]['longitude_latitude'])
    
    ref_length_list.append(jam_ref_data[key]['length'])
    length_list.append(jam_ref_data[key]['length'])
    
    ref_numPts_list.append(float(jam_ref_data[key]['numPts']))
    numPts_list.append(float(jam_ref_data[key]['numPts']))
    
    ref_speed_list.append(jam_ref_data[key]['speed'])
    speed_list.append(jam_ref_data[key]['speed'])


test_loc_list = []
test_length_list = []
test_numPts_list = []
test_speed_list = []
for key in jam_test_data.keys():
    test_loc_list.append(jam_test_data[key]['longitude_latitude'])
    loc_list.append(jam_test_data[key]['longitude_latitude'])
    
    test_length_list.append(jam_test_data[key]['length'])
    length_list.append(jam_test_data[key]['length'])
    
    test_numPts_list.append(float(jam_test_data[key]['numPts']))
    numPts_list.append(float(jam_test_data[key]['numPts']))
    
    test_speed_list.append(jam_test_data[key]['speed'])
    speed_list.append(jam_test_data[key]['speed'])

loc_data = loc_list
length_data = np.array(length_list)
numPts_data = np.array(numPts_list)
speed_data = np.array(speed_list)

In [5]:
len(loc_data), len(ref_loc_list), len(test_loc_list)

(15171, 6995, 8176)

### Quantization levels: loc_k = 3, length_k = 2, numPts_k = 1, speed_k = 2

In [6]:
loc_data = [list(np.reshape(loc_data[i], np.size(loc_data[i]))) for i in range(len(loc_data))]

from scipy.cluster.vq import kmeans2
from ClusterAlg import DF, KMedians

loc_k = 3
num_core = 12
loc_label, loc_centroid, distToClusterCenter = KMedians(loc_data, loc_k, num_core, DF)

In [7]:
length_k = 2
length_centroid, length_label = kmeans2(length_data, length_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

numPts_k = 1
numPts_centroid, numPts_label = kmeans2(numPts_data, numPts_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

speed_k = 2
speed_centroid, speed_label = kmeans2(speed_data, speed_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

In [8]:
distToClusterCenter, loc_label, loc_centroid

([394.08170068724456,
  185.17917640594754,
  317.11956326037216,
  184.684845977834,
  321.0591971239792,
  451.088962870245,
  319.3940546992854,
  168.18172671696877,
  485.65389303284337,
  558.3939964274833,
  836.3885666946322,
  0.0,
  2224.204298319776,
  454.7824535276285,
  1097.6128273239392,
  515.1282167819099,
  603.5386365273631,
  158.8166379845619,
  330.6564129537608,
  1603.3608377922442,
  2887.9990818272386,
  1007.6431646454118,
  2175.149727491413,
  1244.6002990922289,
  1237.504212083224,
  1225.060805704426,
  321.06352318787015,
  1032.4656198917085,
  728.9500977318653,
  1594.0294230879963,
  1543.0540811748103,
  1798.7429613214686,
  2829.062260663455,
  10.45505788789537,
  1108.9480980895407,
  1558.5413312812245,
  1345.5493923273623,
  1256.275716568705,
  2856.5300452109755,
  2122.4200134293023,
  1549.0602646877078,
  861.7249453700952,
  945.9894958842631,
  1093.339359471733,
  2186.549366395631,
  1332.273423931274,
  1701.9789713155028,
  1135.

In [9]:
ref_loc_label = loc_label[0:np.size(np.array(ref_loc_list),0)]
test_loc_label = loc_label[np.size(np.array(ref_loc_list),0):np.size(loc_data,0)]

ref_length_label = length_label[0:np.size(np.array(ref_length_list),0)]
test_length_label = length_label[np.size(np.array(ref_length_list),0):np.size(length_data,0)]

ref_numPts_label = numPts_label[0:np.size(np.array(ref_numPts_list),0)]
test_numPts_label = numPts_label[np.size(np.array(ref_numPts_list),0):np.size(numPts_data,0)]

ref_speed_label = speed_label[0:np.size(np.array(ref_speed_list),0)]
test_speed_label = speed_label[np.size(np.array(ref_speed_list),0):np.size(speed_data,0)]

In [10]:
len(ref_loc_label), len(test_loc_label), len(ref_length_label), len(test_length_label), \
len(ref_numPts_label), len(test_numPts_label), len(ref_speed_label), len(test_speed_label)

(6995, 8176, 6995, 8176, 6995, 8176, 6995, 8176)

In [11]:
quantized_jam_ref_data = {}

i = 0
for key in jam_ref_data.keys():
    data = {'uuid': jam_ref_data[key]['uuid'],
            'longitude_latitude': jam_ref_data[key]['longitude_latitude'],
            'length': jam_ref_data[key]['length'],
            'numPts': float(jam_ref_data[key]['numPts']),
            'speed': jam_ref_data[key]['speed'],
            'startTime': jam_ref_data[key]['startTime'],
            'loc_label': float(ref_loc_label[i]),
            'length_label': float(ref_length_label[i]),
            'numPts_label': float(ref_numPts_label[i]),
            'speed_label': float(ref_speed_label[i])}
    quantized_jam_ref_data[key] = data
    i += 1
    
# Writing JSON data
with open(data_folder + 'quantized_jam_ref_data.json', 'w') as json_file:
    json.dump(quantized_jam_ref_data, json_file)

In [12]:
quantized_jam_test_data = {}

i = 0
for key in jam_test_data.keys():
    data = {'uuid': jam_test_data[key]['uuid'],
            'longitude_latitude': jam_test_data[key]['longitude_latitude'],
            'length': jam_test_data[key]['length'],
            'numPts': float(jam_test_data[key]['numPts']),
            'speed': jam_test_data[key]['speed'],
            'startTime': jam_test_data[key]['startTime'],
            'loc_label': float(test_loc_label[i]),
            'length_label': float(test_length_label[i]),
            'numPts_label': float(test_numPts_label[i]),
            'speed_label': float(test_speed_label[i])}
    quantized_jam_test_data[key] = data
    i += 1
    
# Writing JSON data
with open(data_folder + 'quantized_jam_test_data.json', 'w') as json_file:
    json.dump(quantized_jam_test_data, json_file)